# Домашнее задание №1: линейная регрессия и векторное дифференцирование (10 баллов).

* Максимальное количество баллов за задания в ноутбуке - 11, но больше 10 оценка не ставится, поэтому для получения максимальной оценки можно сделать не все задания.

* Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.

In [203]:
# Вариант

len('Сек') % 4 + 1

4

In [1]:
import numpy as np

## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [213]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

1.4733280321693085e-25


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [45]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))
reg.coef_

2.8178026310899915e-12


array([ 1.50172120e-08,  7.30179686e-09,  2.76451015e-08,  4.17150489e-08,
       -2.59159314e-08, -1.42207128e-09,  1.09796338e-08,  8.93078959e+01,
       -2.97181114e-08,  1.52891166e-08,  3.98966846e-08,  3.73823447e-08,
        2.15398925e-08,  3.54150974e-08,  1.91629716e-10,  1.66049726e-08,
       -1.15313012e-08, -2.41758070e-08,  3.43031208e-08,  7.94576254e-08,
        3.40910477e+01,  2.88356528e+01, -1.77722224e-08,  3.41404640e-08,
       -3.01098008e-09,  4.17853711e-08, -7.24611546e-08, -5.79103231e-09,
       -8.94900562e-09,  2.56378035e-08,  8.11098268e-08, -9.96734504e-09,
        2.91560773e-08,  1.52268432e-08,  1.06144532e-08, -1.32959310e-09,
        1.69004412e-08, -2.00360854e-08, -3.02836042e-08, -3.16503117e-08,
       -8.47234245e-08, -4.26993343e-09, -6.49913832e-09,  5.93787253e-08,
        5.25474384e-08,  2.48623357e-09,  4.04265830e-08, -4.96889900e-08,
       -3.30864395e-09,  9.49693916e+01,  8.00371100e-09,  5.75605024e+01,
       -2.75384742e-09,  

***Задание 1 (0.5 балла).*** Объясните, чем вызвано различие двух полученных значений метрики?

***Задание 2 (0.5 балла).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

### Задание 1 - решение

Разница в метрике MSE, так же, как и разница в коэффициентах получилась из-за того, что градиентный спуск - это стохастический способ нахождения оптмума и, в нем есть некоторая случайность, он может не совпадать с истинным (аналитическим) решением задачи.

Случайность заключается в том, что в этом методе мы постепенно двигаемся шаг за шагом в сторону антиградиента, при этом заданный шаг или по-другому learning rate (гиперпараметр) сильно влияет на решение: слишком большой learning rate $\Rightarrow$ высока вероятность "пропустить" минимум функции, слишком маленький learning rate $\Rightarrow$ алгоритм будет сходиться очень долго, если вовсе сойдется

Когда гиперпармаетры подобраны хорошо, стохастически найденное решение должно быть близко к аналитическому, но в этом случае разница кажется существенной, поэтому перейдем к заданию 2

### Задание 2 - решение

Идея в том, чтобы кроссвалидацией подобрать оптимальное значение гиперпараметра alpha

Чтобы получить минимальное различие MSE с аналитическим решением - я выбрал параметр scoring = 'neg_mean_squared_error'

- В linspace немного отступил от нуля, потому что оптимизатор "застревал" в нуле и показывал, что оптимальная альфа равна нулю
- Сначала я взял модель со всеми дефолтными параметрами, обучил ее и потом начал оптимизировать MSE по alpha кроссвалидацией
- Получил оптимальную alpha, вставил этот оптимальный гиперпараметр в SGDRegressor, сделал predict и получил MSE
- Полученный MSE методом градиентного бустинга совпадает порядок (e-25), что показывает, что у нас очень близкие MSE к аналитическому решению

Если запускать код несколько раз, использую полученный оптимальный гиперпараметр - MSE будет изменяться, т.к. метод стохастический, а не аналитический, но при этом порядок MSE всегда остается таким же, как и в аналитическом методе (e-25)

Ответ: 1e-19

In [115]:
reg = SGDRegressor().fit(X, y)

In [117]:
# давайте используем кросс-валидацию для поиска оптимальных гиперпараметров

from sklearn.model_selection import GridSearchCV

alphas = np.linspace(0.0000000000000000001,1,100)

params = {'alpha':alphas}
num_splits = 3

#print(params)
cv = GridSearchCV(reg,
                  params,
                  scoring='neg_mean_squared_error',
                  cv=num_splits
                 )

cv.fit(X, y)

print(cv.best_params_)

{'alpha': 1e-19}


In [133]:
reg = SGDRegressor(alpha=cv.best_params_['alpha']).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

3.6912224189998502e-25


## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс. 

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и второй варианты) или l2 (третий и четвертый варианты) регуляризацию. 

### Задание 3 - решение

Не могу найти ошибку - не проходит assertion error

In [214]:
import statsmodels.api as sm
from tqdm import tqdm

In [326]:
class LinearRegression(object):
    def __init__(self, alpha=0.45, l_ratio=0.00001, tol=0.0001, max_iter=10000):
        '''
        Для начала необходимо инициализировать параметры
        alpha - это learning rate или шаг обучения
        l_ratio - параметр регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        '''
        
        self.alpha = alpha
        self.l_ratio = l_ratio
        self.tol = tol
        self.max_iter = max_iter
        

             
    def fit(self, X, y):
        '''
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        '''
        # Добавим столбец констант к матрице признаков
        X = sm.add_constant(X)
        
        # Количество наблюдений        
        N = len(y)
        # Количество предикторов        
        K = X.shape[1]

        
        # Создадим пустую матрицу весов (пусть все веса сначала равны 1)        
        W = np.zeros(K)
        
        for i in tqdm(range(self.max_iter)):
            
            # each iteration y will change, so good to have temporary variable for y            
            temp_y = X @ W
            
            # MSE loss
            loss_func_t_0 = 1/N * sum((y - temp_y)**2)
            
            # define gradinent in a matrix form
            W_grad = 2/N * (X.T @ ((X @ W) - y))
            
            # update weights
            W = W - self.alpha * W_grad
            
            # update temporary y
            y_temp = X @ W
            
            # MSE loss in the next period iteration            
            loss_func_t_1 = 1/N * sum((y - temp_y)**2)
            
            # Идея здесь сравнить то, насколько функционалы до и после обновления весов отличаются 
            # Вернее насколько велик модуль их разности, который для моего варианта служит критерием останова            
               
            loss_diff = np.linalg.norm(loss_func_t_1 - loss_func_t_0)
            
            if loss_diff >= self.tol:
                continue
            else:
                break
        
        # нужно, чтобы использовать в предикт функции       
        self.Weights = W
        
        return self


    def predict(self, X):
        '''
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        '''
        # Добавим столбец констант к матрице признаков
        X = sm.add_constant(X)
        
        # Прогноз это произведение матриц предикторов и полученных из функции fit весов        
        Y_pred = X @ self.Weights
        
        return Y_pred

In [327]:
my_reg = LinearRegression()
my_reg.fit(X, y)

my_reg.predict(X)
# assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
# print('You are amazing! Great work!')

  0%|          | 0/10000 [00:00<?, ?it/s]


array([121.31125986,  -1.8284482 ,   1.72599181, ...,  44.80971777,
       112.0646511 ,  29.44430206])

In [311]:
y

array([125.05840824,  -3.45110706, -22.24543605, ...,  64.49833094,
       147.81654533,  34.37837112])

***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки

* с l1-регуляризацией (from sklearn.linear_model import Lasso, **первый и второй вариант**) или с l2-регуляризацией (from sklearn.linear_model import Ridge, **третий и четвертый вариант**)
* со значением параметра регуляризации **0.1 - для первого и третьего варианта, 0.01 - для второго и четвертого варианта**. 

Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

### Задание 5 - решение

In [200]:
# у меня 4й вариант
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

Ridge_reg = Ridge(alpha=0.01)
Ridge_reg.fit(X, y)

# Check MSE values
mse = cross_val_score(Ridge_reg, X, y, cv=5, scoring="neg_mean_squared_error")
mse

# Сравнивать не с чем)))

array([-3.47199278e-08, -3.27115335e-08, -3.54852340e-08, -3.80873109e-08,
       -3.41946709e-08])

***Задание 6* (1 балл).***
Пусть $P, Q \in \mathbb{R}^{n\times n}$. Найдите $\nabla_Q tr(PQ)$

### Задание 6 - решение

$\nabla_Q tr(PQ) = \frac{\partial}{\partial Q_{i,j}} tr(PQ) = \frac{\partial}{\partial Q_{i,j}} \sum_k (PQ)_{kk} = \frac{\partial}{\partial Q_{i,j}} \sum_{k,l} P_{kl}Q_{lk} = P_{ji} $

Следовательно, мы можем получить следующий результат: $\nabla_Q tr(PQ) = P^T$

***Задание 7* (1 балл).***
Пусть $x, y \in \mathbb{R}^{n}, M \in \mathbb{R}^{n\times n}$. Найдите $\nabla_M x^T M y$

### Задание 7 - решение

$\nabla_M x^T M y = \nabla_M TR(x^T M y) $ - потому что произведение в скобках дает число

$\nabla_M TR(x^T M y) = \nabla_M TR(M y x^T)$ - по циклическому свойству следа матрицы

Из решения задачи 6 мы можем взять производную промежуточного результата $\nabla_M TR(M y x^T) = xy^T$


Решения заданий 6 и 7 можно написать на листочке и отправить в anytask вместе с заполненным ноутбуком.